In [1]:
! pip install qiskit

In [2]:
! pip install qiskit-machine-learning

In [3]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from qiskit import *

# External imports
from pylab import cm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
#from qiskit.algorithms.optimizers import SPSA

from qiskit.circuit.library import PauliFeatureMap
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data

#from qiskit.utils import QuantumInstance
#from qiskit_machine_learning.kernels import QuantumKernel

import random
import time
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
df = pd.read_csv('Thesis MCS Final Dataset.csv')
df.head()

,Date,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,27-06-2023,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,27-06-2023,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,27-06-2023,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,27-06-2023,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,27-06-2023,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0


In [5]:
df.drop(columns=['Date'], inplace=True)
#df.drop(columns=['Temperature(C)'], inplace=True)
df

,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,15,1014,Clear,Clear,0,0,1,0
4500,20:00,19,19,1,80,14.0,13,1015,Clear,Clear,0,0,1,0
4501,21:00,19,19,2,82,14.0,13,1015,Clear,Clear,0,0,1,0
4502,22:00,18,18,2,83,14.0,12,1015,Clear,Clear,0,0,1,0


In [6]:
df.dropna(inplace=True)
df

,Time,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Predicted_State,Real State,Rain,Cloudy,Sunny,Fog
0,08:00,26,31,62,91,4.0,25,1003,Mostly Cloudy,Cloudy,0,1,0,0
1,09:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Cloudy,0,1,0,0
2,10:00,27,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
3,11:00,28,33,66,88,4.0,25,1004,Mostly Cloudy,Rain,1,1,0,0
4,12:00,29,36,65,93,1.6,28,1005,Cloudy,Sunny,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,19:00,20,20,1,77,8.0,15,1014,Clear,Clear,0,0,1,0
4500,20:00,19,19,1,80,14.0,13,1015,Clear,Clear,0,0,1,0
4501,21:00,19,19,2,82,14.0,13,1015,Clear,Clear,0,0,1,0
4502,22:00,18,18,2,83,14.0,12,1015,Clear,Clear,0,0,1,0


In [7]:
df.drop(columns=['Time', 'Predicted_State','Real State','Cloudy','Sunny'], inplace=True)
df

,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%),Visibity(km),Dew Point,Pressure(hPa),Rain,Fog
0,26,31,62,91,4.0,25,1003,0,0
1,27,33,66,88,4.0,25,1004,0,0
2,27,33,66,88,4.0,25,1004,1,0
3,28,33,66,88,4.0,25,1004,1,0
4,29,36,65,93,1.6,28,1005,0,0
...,...,...,...,...,...,...,...,...,...
4499,20,20,1,77,8.0,15,1014,0,0
4500,19,19,1,80,14.0,13,1015,0,0
4501,19,19,2,82,14.0,13,1015,0,0
4502,18,18,2,83,14.0,12,1015,0,0


In [8]:
df.drop(columns=['Fog'], inplace=True)
df.drop(columns=['Dew Point','Visibity(km)','Pressure(hPa)'], inplace=True)

In [9]:
from qiskit import *

In [10]:
print("Using 3 features Price, Open, High: ")
sz = []
svm_acc = []
qsvm_acc = []
for data_size in range(50, 300, 50):
    print("for size", data_size)
    sz.append(data_size)
    x = df.iloc[:data_size, 0:-1]
    y = df.iloc[:data_size, -1]
    # spltting the dataset into train and test set
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
    from sklearn import svm
    svc_clf = svm.SVC(random_state = 7)
    svc_clf.fit(x_train, y_train)
    svc_score = svc_clf.score(x_test, y_test)
    print(f"svm classification score: {svc_score}")
    svm_acc.append(svc_score)
    import qiskit
    import matplotlib.pyplot as plt
    import numpy as np

    from sklearn.svm import SVC
    from sklearn.cluster import SpectralClustering
    from sklearn.metrics import normalized_mutual_info_score

    #from qiskit import BasicAer
    from qiskit.circuit.library import ZZFeatureMap
    #from qiskit.utils import QuantumInstance, algorithm_globals
    from qiskit_machine_learning.algorithms import QSVC
    #from qiskit_machine_learning.kernels import QuantumKernel
    #from qiskit_machine_learning.datasets import ad_hoc_data

    seed = 12345
    #algorithm_globals.random_seed = seed
    # Building Kernel
    dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

    

    #dse_kernel = QuantumKernel(feature_map=dse_feature_map)
    dse_svc = QSVC(kernel=dse_feature_map)
    dse_svc.fit(x_train, y_train)
    dse_score = dse_svc.score(x_test, y_test)

    print(f"qsvm classification test score: {dse_score}")
    qsvm_acc.append(dse_score)

Using 3 features Price, Open, High: 
for size 50
svm classification score: 0.46153846153846156


/tmp/ipykernel_1398/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.6153846153846154
for size 100
svm classification score: 0.68


/tmp/ipykernel_1398/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.68
for size 150
svm classification score: 0.6842105263157895


...

qsvm classification test score: 0.6578947368421053
for size 200
svm classification score: 0.84


/tmp/ipykernel_1398/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.82
for size 250
svm classification score: 0.7619047619047619


/tmp/ipykernel_1398/1918751165.py:42: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.746031746031746


In [11]:
x

,Temperature(C),Feels Like(C),Chance of Prec.(%),Humidity(%)
0,26,31,62,91
1,27,33,66,88
2,27,33,66,88
3,28,33,66,88
4,29,36,65,93
...,...,...,...,...
245,30,36,30,73
246,31,38,59,71
247,31,37,20,74
248,31,37,2,75


In [12]:
y

0      0
1      0
2      1
3      1
4      0
      ..
245    1
246    0
247    0
248    0
249    0
Name: Rain, Length: 250, dtype: int64

In [13]:
print(svm_acc)
print(qsvm_acc)

[0.46153846153846156, 0.68, 0.6842105263157895, 0.84, 0.7619047619047619]
[0.6153846153846154, 0.68, 0.6578947368421053, 0.82, 0.746031746031746]


In [14]:

sz = [50,100,150,200,250]

import random
total = 5
for cnt in range(total):
    print("current iteration: ", cnt+1)
    for idx in range(len(sz)):
        data_size = sz[idx]
        print("for size", data_size)
        st = random.randint(0, 300-data_size)
        x = df.iloc[st:st+data_size, 0:-1]
        y = df.iloc[st:st+data_size, -1]
        # spltting the dataset into train and test set
        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 31)
        from sklearn import svm
        svc_clf = svm.SVC(random_state = 7)
        svc_clf.fit(x_train, y_train)
        svc_score = svc_clf.score(x_test, y_test)
        print(f"svm classification score: {svc_score}")
        svm_acc[idx] = svm_acc[idx] + svc_score
        import qiskit
        import matplotlib.pyplot as plt
        import numpy as np

        from sklearn.svm import SVC
        from sklearn.cluster import SpectralClustering
        from sklearn.metrics import normalized_mutual_info_score

        #from qiskit import BasicAer
        from qiskit.circuit.library import ZZFeatureMap
        #from qiskit.utils import QuantumInstance, algorithm_globals
        from qiskit_machine_learning.algorithms import QSVC
        #from qiskit_machine_learning.kernels import QuantumKernel
        #from qiskit_machine_learning.datasets import ad_hoc_data

        seed = 12345
        #algorithm_globals.random_seed = seed
        # Building Kernel
        dse_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")



        #dse_kernel = QuantumKernel(feature_map=dse_feature_map)
        dse_svc = QSVC(kernel=dse_feature_map)
        dse_svc.fit(x_train, y_train)
        dse_score = dse_svc.score(x_test, y_test)

        print(f"qsvm classification test score: {dse_score}")
    #qsvm_acc.append(dse_score)
        
        qsvm_acc[idx] = qsvm_acc[idx] + dse_score
    print("=============================================")

current iteration:  1
for size 50
svm classification score: 1.0


/tmp/ipykernel_1398/814570115.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 1.0
for size 100
svm classification score: 0.88


/tmp/ipykernel_1398/814570115.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.88
for size 150
svm classification score: 0.7894736842105263


...

qsvm classification test score: 0.8157894736842105
for size 200
svm classification score: 0.84


/tmp/ipykernel_1398/814570115.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.84
for size 250
svm classification score: 0.6984126984126984


/tmp/ipykernel_1398/814570115.py:45: QiskitMachineLearningWarning: "'kernel' argument is not supported and will be discarded, please use 'quantum_kernel' instead."
  dse_svc = QSVC(kernel=dse_feature_map)


qsvm classification test score: 0.7142857142857143


In [15]:
print(svm_acc, qsvm_acc)

[4.384615384615385, 5.0, 4.657894736842105, 4.66, 4.349206349206349] [4.461538461538462, 5.0, 4.657894736842105, 4.640000000000001, 4.333333333333333]


In [16]:
for idx in range(len(sz)):
    print(idx, svm_acc[idx]/6, '\t\t', qsvm_acc[idx]/6)

0 0.7307692307692308 		 0.7435897435897436
1 0.8333333333333334 		 0.8333333333333334
2 0.7763157894736842 		 0.7763157894736842
3 0.7766666666666667 		 0.7733333333333334
4 0.7248677248677248 		 0.7222222222222222
